In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix


In [ ]:
# Load your restaurant review dataset (replace 'data.csv' with your dataset file)
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Restaurant_Reviews.tsv', delimiter='\t', quoting=3)

In [ ]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data['Review'], data['Liked'], test_size=0.2, random_state=42)


In [ ]:
# Tokenize and pad the text data
max_words = 5000  # Maximum number of words in your vocabulary
max_sequence_length = 100  # Maximum sequence length (you can adjust this)
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)


In [ ]:
# Encode the labels (convert labels to numerical values)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)


In [ ]:
# Build an RNN model
embedding_dim = 100  # You can adjust this dimension
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SimpleRNN(units=128))  # You can adjust the number of units
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
epochs = 5  # You can adjust the number of epochs
batch_size = 64  # You can adjust the batch size
model.fit(X_train_pad, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad, y_test_encoded))


Epoch 1/5
13/13 [==============================] - 4s 144ms/step - loss: 0.6980 - accuracy: 0.5312 - val_loss: 0.7138 - val_accuracy: 0.4800
Epoch 2/5
13/13 [==============================] - 1s 103ms/step - loss: 0.6532 - accuracy: 0.6050 - val_loss: 0.6828 - val_accuracy: 0.5750
Epoch 3/5
13/13 [==============================] - 1s 81ms/step - loss: 0.5227 - accuracy: 0.8938 - val_loss: 0.6821 - val_accuracy: 0.5650
Epoch 4/5
13/13 [==============================] - 1s 61ms/step - loss: 0.2976 - accuracy: 0.9413 - val_loss: 0.7303 - val_accuracy: 0.5800
Epoch 5/5
13/13 [==============================] - 1s 60ms/step - loss: 0.1084 - accuracy: 0.9787 - val_loss: 0.7865 - val_accuracy: 0.6200


In [ ]:
# Evaluate the model
y_pred = np.argmax(model.predict(X_test_pad), axis=-1)  # Convert softmax probabilities to class labels
accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred, average='weighted')
recall = recall_score(y_test_encoded, y_pred, average='weighted')
confusion = confusion_matrix(y_test_encoded, y_pred)


7/7 [==============================] - 1s 22ms/step


In [ ]:
print('RECURRENT NEURAL NETWORK (RNN):')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n",confusion)

RECURRENT NEURAL NETWORK (RNN):
Accuracy: 0.62
Precision: 0.6219871794871795
Recall: 0.62
Confusion Matrix:
 [[62 34]
 [42 62]]
